<a href="https://colab.research.google.com/github/skanderbenmansour/nlp_study_group/blob/master/nina/movie_reviews_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install skorch

     |████████████████████████████████| 122kB 2.8MB/s 


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import re
from random import shuffle

from skorch import NeuralNetClassifier
from sklearn.datasets import make_classification

torch.manual_seed(1)

# Embeddings

## Load and preprocess

In [0]:
path_pos = '/content/drive/My Drive/Datasets/review_polarity/txt_sentoken/pos'
path_neg = '/content/drive/My Drive/Datasets/review_polarity/txt_sentoken/neg'

In [0]:
def preprocess(doc):
    return [re.sub('[^\w]|[\d]','', word.lower()) for word in doc]

def remove_words(vocab):
    return [i for i in vocab if i not in ['', 'a','it','the', 'i']]

def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

def make_ngram_vector(sentence, word_to_ix, n=3):
    ngram_vectors = []
    for i in range(len(sentence) - n):
        curr_ngram = [[], None]     
        for j in range(n):
            if j == n-1:
                curr_ngram[1] = sentence[i + j]
            else:
                curr_ngram[0].append(sentence[i + j])
        ngram_vectors.append(curr_ngram)

    return ngram_vectors

In [5]:
import glob

data = []

# load data in
for path in glob.glob(path_pos+'/*.txt'):
  with open(path, 'r') as f:
    data.append((preprocess(f.read().replace('\n', ' ').split()), "P"))

print('finished positive reviews')

for path in glob.glob(path_neg+'/*.txt'):
  with open(path, 'r') as f:
    data.append((preprocess(f.read().replace('\n', ' ').split()), "N"))

print('finished negative reviews')

finished positive reviews
finished negative reviews


## Transform sentences

In [6]:
data[0][0][1]

'am'

In [0]:
import numpy as np

data_text = np.array([np.array(text)[:50] for text, label in data])

In [0]:
data_text = np.concatenate(data_text)

In [26]:
set("hello there".split())

{'hello', 'there'}

In [0]:
vocab = set(remove_words(data_text))
word_to_ix = {word: i for i, word in enumerate(vocab)}

# triples
ngram = []
X_ngram = []

ngram = make_ngram_vector(remove_words(data_text), word_to_ix)

In [0]:
X = [make_context_vector(i, word_to_ix) for i,j in ngram]
y = [torch.tensor([word_to_ix[j]], dtype=torch.long) for i,j in ngram ]

In [29]:
X[1], y[1]

(tensor([5478, 5480]), tensor([1498]))

# Model

In [0]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
VOCAB_SIZE = len(vocab)


class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        # print(inputs.shape)
        embeds = self.embeddings(inputs).view((-1, 20))
        out = self.linear1(embeds)
        out = F.relu(out)
        out = self.linear2(out)
        log_probs = F.softmax(out, dim=1)
        return log_probs


net_ngram = NeuralNetClassifier(
    NGramLanguageModeler(VOCAB_SIZE, EMBEDDING_DIM, CONTEXT_SIZE),
    max_epochs=20,
    lr=0.1,
    optimizer=optim.SGD,
    optimizer__lr=0.01,
    #train_split=None,
    # Shuffle training data on each epoch
    iterator_train__shuffle=False,
)

In [31]:
torch.stack(X).squeeze().shape

torch.Size([76806, 2])

In [0]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(torch.stack(X).squeeze(), torch.stack(y).squeeze(), test_size=0.25, random_state=42)

In [33]:
# X_train.shape, y_train.shape

(torch.Size([57604, 2]), torch.Size([57604]))

# Train

In [0]:
X_e = torch.stack(X).squeeze()
y_e = torch.stack(y).squeeze()

In [35]:
X_e.shape, y_e.shape

(torch.Size([76806, 2]), torch.Size([76806]))

In [36]:
net_ngram.fit(X_e, y_e)  

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


  epoch    train_loss    valid_acc    valid_loss      dur
-------  ------------  -----------  ------------  -------
      1        9.3681       0.0318        9.2758  26.2587
      2        9.1761       0.0329        9.0737  32.1723
      3        8.9557       0.0329        8.8415  31.9417
      4        8.7282       0.0324        8.6360  31.6852
      5        8.5474       0.0316        8.4833  31.5739
      6        8.4090       0.0324        8.3587  31.5368
      7        8.2891       0.0329        8.2466  31.5278
      8        8.1824       0.0338        8.1500  31.7271
      9        8.0929       0.0332        8.0713  31.5961
     10        8.0199       0.0346        8.0074  31.8888
     11        7.9594       0.0356        7.9542  31.8606
     12        7.9077       0.0356        7.9083  31.9794
     13        7.8621       0.0360        7.8679  36.3240
     14        7.8212       0.0361        7.8320  32.7184
     15        7.7845       0.0365        7.8000  31.7844
     16       

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=NGramLanguageModeler(
    (embeddings): Embedding(12359, 10)
    (linear1): Linear(in_features=20, out_features=128, bias=True)
    (linear2): Linear(in_features=128, out_features=12359, bias=True)
  ),
)